In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 1) Développer une interface qui permet d’introduire vos données (images ou vidéo)
# Assurez-vous d'avoir un dossier "data" avec les sous-dossiers pour chaque classe d'image (par exemple : explosif, inflammable, toxique, etc.)
data_dir = 'data'
batch_size = 32
img_height, img_width = 150, 150

# Créez des générateurs d'images pour l'entraînement et la validation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# 2) Extraire les objets à classifier/interpréter/reconnaître à partir des données originales.
# 3) Caractériser chaque objet en utilisant des mesures descriptives appropriées.
# 4) Implémenter et tester l’algorithme de classification de formes.
# Construire le modèle
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Entraîner le modèle
epochs = 20
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

# Enregistrer le modèle entraîné
model.save('pictogrammes_dangereux.h5')

# Afficher les courbes d'apprentissage
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(range(epochs), history.history['loss'], label='Perte (entraînement)')
plt.plot(range(epochs), history.history['val_loss'], label='Perte (validation)')
plt.xlabel('Ésure') # Ajoute l'étiquette 'Époque' à l'axe des x
plt.ylabel('Perte') # Ajoute l'étiquette 'Perte' à l'axe des y
plt.legend() # Ajoute la légende

plt.subplot(1, 2, 2)
plt.plot(range(epochs), history.history['accuracy'], label='Précision (entraînement)')
plt.plot(range(epochs), history.history['val_accuracy'], label='Précision (validation)')
plt.xlabel('Époque') # Ajoute l'étiquette 'Époque' à l'axe des x
plt.ylabel('Précision') # Ajoute l'étiquette 'Précision' à l'axe des y
plt.legend() # Ajoute la légende
plt.show()
